<a href="https://colab.research.google.com/github/jdu8/underage-detection/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Sep 28 17:03:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!mkdir kaggle
!mv kaggle.json kaggle/
!chmod 600 kaggle/kaggle.json

In [7]:
!kaggle datasets download -d abhikjha/utk-face-cropped


100% 232M/232M [00:04<00:00, 87.9MB/s]
100% 232M/232M [00:04<00:00, 59.3MB/s]


In [ ]:
!unzip utk-face-cropped.zip

In [2]:
import os
import cv2
from tqdm import tqdm
import numpy as np

In [ ]:
PATH='utkcropped/'
x=[]
y=[]
for i in tqdm(os.listdir(PATH)):
  if i!='utkcropped':
    age=int(i.split("_")[0])
    temp_img=cv2.imread(PATH+i)
    x.append(temp_img/255.0)
    if age<18:
      y.append(1)
    else:
      y.append(0)
x=np.array(x)
y=np.array(y)
# x=x/255.0
print(f"Images Shape: {x.shape}")
print(f"Labels Shape: {y.shape}")

 53%|█████▎    | 12595/23710 [00:22<01:34, 117.28it/s]

In [ ]:
np.random.seed(69)
np.random.shuffle(x)
np.random.seed(69)
np.random.shuffle(y)

In [ ]:
x_train=x[:int(len(x)*0.8)]
x_test=x[int(len(x)*0.8):]
y_train=y[:int(len(x)*0.8)]
y_test=y[int(len(x)*0.8):]
print(f"X train Shape: {x_train.shape}")
print(f"X text Shape: {x_test.shape}")
print(f"Y train Shape: {y_train.shape}")
print(f"Y test Shape: {y_test.shape}")

In [ ]:
import keras 
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [ ]:
model = keras.models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = keras.losses.binary_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(x_train,y_train,epochs=20,batch_size=128,validation_data=(x_test,y_test))